<a href="https://colab.research.google.com/github/punitarani/MAT-494/blob/master/3.5-6%20K-Means%20and%20SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 3.5-6 K-Means and Support Vector Machine

## K-Means

K-Means is an unsupervised learning algorithm that is used to cluster data.

Within-Cluster Sum of Squares (WCSS)
$\text{WCSS}_i = \sum_{x \in S_i} {\left\| x_i - \mu_j \right\|^2}$

The algorithm is defined as:

1. Initialize $k$ centroids $\mu_1, \dots, \mu_k \in \mathbb{R}^d$
2. Repeat until convergence:
    1. Assign each sample $x_i$ to the closest centroid $\mu_j$:
        $j = \arg \min \sum_{i=1}^{k}{WCSS}$
    2. Update the centroids:
        $\mu(S) = \frac{1}{|S|} \sum_{x \in S} x$

where:
- $k$ is the number of clusters
- $n$ is the number of features
- $x_i$ is the $i$-th sample
- $\mu_j$ is the $j$-th centroid
- $S$ is the set of samples assigned to the $j$-th cluster

K-Means converges by $\sum_{i=1}^{k}{WCSS}$ monotonically decreasing in each iteration.
